In [1]:
import t3f
import tensorflow as tf
import numpy as np
from environments import Snake
from methods import ReplayMemory

import matplotlib.pyplot as plt
%matplotlib inline

/Users/mathemage/anaconda3/envs/tf/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/mathemage/anaconda3/envs/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
w = 4
h = 4

batch_size = 256

In [3]:
train_env = Snake(grid_size=(w, h))

In [4]:
state_shape = (w * h) * [4, ]

In [5]:
shape = (w * h) * [4,] + [4, ]

In [6]:
q0init = t3f.cast(t3f.random_tensor(shape=shape, stddev=0.001, tt_rank=8), tf.float64)

In [7]:
q0 = t3f.get_variable('Q', initializer=q0init)

In [8]:
input_states = tf.placeholder(tf.int32, [batch_size, w, h, 1])
input_actions = tf.placeholder(tf.int32, [batch_size, 1])
input_rewards = tf.placeholder(tf.float64, [batch_size, ])

input_states_ = tf.placeholder(tf.int32, [batch_size, w, h, 1])


In [17]:
input_s_and_a = tf.concat([tf.reshape(input_states, (-1, w * h)), input_actions], axis=1) 

In [18]:
q = get_q_action_values(q0, input_s_and_a)

In [19]:
input_s_ = tf.reshape(input_states_, (-1, w, h))

q_ = tf.reduce_max(get_q_values(q0, input_s_), axis=1)

q_select = tf.argmax(get_q_values(q0, input_s_), axis=1)

In [22]:
targets = gamma * q_ + input_rewards


In [23]:
loss = tf.losses.huber_loss(q, targets)

In [24]:
optimizer = tf.train.AdamOptimizer(2.5e-4)

step = optimizer.minimize(loss)

In [25]:
def get_actions(s, qtt, sess):
    feed_dict = {input_states_:s}
    return sess.run(q_select, feed_dict=feed_dict)

In [26]:
def tf_repeat(x, num):
    u = tf.reshape(x, (-1, 1))
    u = tf.tile(u, tf.TensorShape([tf.Dimension(1), num]))
    u = tf.reshape(u, (-1, ))
    return u

def get_q_values(qtt, s):
    # s [None x w x h]
    
    s_shape = s.shape[1:].as_list()
    
    b = s.shape[0]
    w = s_shape[0]
    h = s_shape[1]
    s_idx = tf.reshape(s, (-1, h * w))    
    s_a_idx = tf.concat(4 * [s_idx], axis=0) 
    a_idx = tf.reshape(tf_repeat(tf.range(start=0, limit=4), b), (-1, 1)) 
    s_a_idx = tf.concat([s_a_idx, a_idx], axis=1)
    vals = t3f.gather_nd(qtt, s_a_idx, dtype=tf.float64)
    return tf.transpose(tf.reshape(vals, shape=(4, b)))


def get_q_action_values(qtt, sa):
    return t3f.gather_nd(qtt, sa, dtype=tf.float64)

In [27]:
replay_memory_size=50000
replay_start_size=10000
init_eps=1
final_eps=0.1
annealing_steps=100000
gamma=0.99
max_episode_length=2000
num_episodes = 1000000

In [28]:
rep_buffer = ReplayMemory(replay_memory_size)
frame_count = 0
while (frame_count < replay_start_size):
    s = train_env.reset()
    for time_step in range(max_episode_length):
        a = np.random.randint(4)
        s_, r, end = train_env.step(a)
        rep_buffer.push(s, a, np.sign(r), s_, end)
        s = s_
        frame_count += 1
        if end:
            break

In [29]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [30]:
s = train_env.reset()

In [31]:
sx = s.astype(np.int32)

In [32]:
train_rewards = []
for i in range(num_episodes):
    s = train_env.reset()
    R = 0
    for time_step in range(max_episode_length):
        a = get_actions([s] * batch_size, q0, sess)[0]
        s_, r, done = train_env.step(a)
        rep_buffer.push(s, a, np.sign(r), s_, end)
        s = s_
        R += r
        if time_step % 4 == 0:
            batch = rep_buffer.get_batch(batch_size)
            sess.run(step, feed_dict={input_states:batch.s,
                                      input_actions:batch.a.reshape((-1, 1)),
                                      input_rewards:batch.r,
                                      input_states_:batch.s_})
        if done: break
    train_rewards.append(R) 
    if i % 1000 == 0:
        avg_reward = np.mean(train_rewards[-1000:])
        print("Train info:", avg_reward ) 

Train info: -1.0
Train info: -0.885
Train info: -0.883
Train info: -0.787
Train info: -0.715
Train info: -0.529
Train info: -0.438


KeyboardInterrupt: 

In [70]:
rep_buffer.get_batch(32).s_.shape

(32, 4, 4, 1)